## Imports
We have 3 important files for this purpose and each and every dependency, class, import, function, variable etc is being imported from these modules

1. `detection_helpers` which I made to wrap the original `YOLOv-7` code along with helper functions
2. `tracking_helpers` has modular code which is used to wrap the `DeepSORT` repo and workings
3. `bridge_wrapper` acts as a bridge to bind **ANY** detection model with `DeepSORT`

In [ ]:
!git clone https://github.com/nwojke/deep_sort.git

In [1]:
from detection_helpers import *
from tracking_helpers import *
from  bridge_wrapper import *
from PIL import Image

# Detection
Detect objects using `Yolov-7`

In [2]:
detector = Detector(classes = [0,17,32]) # it'll detect ONLY [person,horses,sports ball]. class = None means detect all classes. List info at: "data/coco.yaml"
detector.load_model('./weights/yolov7x.pt',) # pass the path to the trained weight file


# # Pass in any image path or Numpy Image using 'BGR' format
# result = detector.detect('./IO_data/input/images/horses.jpg', plot_bb = True) # plot_bb = False output the predictions as [x,y,w,h, confidence, class]


# if len(result.shape) == 3:# If it is image, convert it to proper image. detector will give "BGR" image
#     result = Image.fromarray(cv2.cvtColor(result,cv2.COLOR_BGR2RGB)) 
    
# result

Fusing layers... 
 Convert model to Traced-model... 


d:\Python3.10\lib\site-packages\torch\nn\modules\module.py:831: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten\src\ATen/core/TensorBody.h:491.)
  if param.grad is not None:


 traced_script_module saved! 
 model is traced! 



d:\Python3.10\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


# Tracking

Works as follows:
1. Read each frame of video using `OpenCV`
2. Get Bounding Box or Detections from the model per frame
3. Crop those patches and pass on to `reID` model for re identification which is a part of `DeepSORT` method
4. Get the above embeddings and then use `Kalman Filtering` and `Hungerian assignment` to assign the correct BB to the respective object
5. Show, Save the frames

In [3]:
# Initialise  class that binds detector and tracker in one class
tracker = YOLOv7_DeepSORT(reID_model_path="./deep_sort/model_weights/mars-small128.pb", detector=detector)

# output = None will not save the output video
result = tracker.track_video("./IO_data/input/video/s50.mp4", output="./IO_data/output/s50.avi", show_live = True, skip_frames = 0, count_objects = True, verbose=2)

Processed frame no: 1 || Current FPS: 0.87 || Objects tracked: 14
Processed frame no: 2 || Current FPS: 2.76 || Objects tracked: 14
Tracker ID: 1, Class: person,  BBox Coords (xmin, ymin, xmax, ymax): (234, 251, 262, 303)
Tracker ID: 2, Class: person,  BBox Coords (xmin, ymin, xmax, ymax): (159, 228, 205, 294)
Tracker ID: 3, Class: person,  BBox Coords (xmin, ymin, xmax, ymax): (518, 187, 551, 250)
Tracker ID: 4, Class: person,  BBox Coords (xmin, ymin, xmax, ymax): (387, 190, 406, 235)
Tracker ID: 5, Class: person,  BBox Coords (xmin, ymin, xmax, ymax): (125, 205, 166, 271)
Tracker ID: 6, Class: person,  BBox Coords (xmin, ymin, xmax, ymax): (197, 135, 225, 176)
Tracker ID: 7, Class: person,  BBox Coords (xmin, ymin, xmax, ymax): (219, 96, 232, 138)
Tracker ID: 8, Class: person,  BBox Coords (xmin, ymin, xmax, ymax): (560, 122, 575, 161)
Tracker ID: 11, Class: person,  BBox Coords (xmin, ymin, xmax, ymax): (194, 121, 206, 153)
Tracker ID: 12, Class: person,  BBox Coords (xmin, ymin, x

In [7]:
import pandas as pd

pd.DataFrame(result).to_csv("./IO_data/output/s50.csv", index=False)